### Práctica de Formatos de datos: XML

El objetivo de esta práctica es crear una aplicación simple para llegar determinados edificios monumentales de Madrid. Para ello se van a utilizar dos APIS diferentes ofrecidas por el portal de datos abiertos del ayuntamiento de Madrid.

En primer lugar se va a usar el catálogo "Edificios de carácter monumental". Este catálogo ofrece una relación de edificios artísticos de la ciudad de Madrid, con datos de dirección y de localización para georeferenciarlos. No se trata de una relación exhaustiva, sino de aquellos monumentos y edificios de los cuales ya figura información en la web municipal. El siguiente trozo de código permite recuperar el archivo xml en cuestion, y lo almacena en disco.

In [9]:
import urllib.request
from xml.etree import ElementTree
#Crea el arbol ElementTree
def createTree():
    x = urllib.request.urlopen('https://datos.madrid.es/egob/catalogo/208844-0-monumentos-edificios.xml')
    a = ElementTree.parse(x)
    return a

__[Ejercicio 1 [3 puntos]]__

Se pide mostrar al usuario un listado de los nombres de los monumentos que contiene el archivo con el objetivo de que elija uno de los monumentos. Para facilitar la selección del monumento se puede asignar a cada monumento un número. Fíjate que el nombre que se quiere recuperar es aquel que aparece dentro de las etiquetas <atributo nombre="NOMBRE">. Por ejemplo en el caso del "Banco de España" sería:
    
    <contenido>
        <tipo>EntidadesYOrganismos</tipo>
        <atributos idioma="es">
            <atributo nombre="ID-ENTIDAD">13947</atributo>
            <atributo nombre="NOMBRE">Banco de España</atributo>
            
A continuación se le mostrará por pantalla toda la información acerca del monumento de una manera amigable (es decir sin etiquetas).

In [10]:
def displayAndList(arbol):
    lista = []
    i = 1
    for nodo in arbol.iter(): #Todos se llaman atributo
        if nodo.attrib.get("nombre") == "NOMBRE":
            print(str(i) + "\t" + nodo.text)
            lista.append(nodo.text)
            i = i + 1 
    return lista

In [11]:
def checkCorrectness(lista, caso):
    inputCorrecto = False
    while not inputCorrecto:
        elemento = input('\nIntroduce el numero del '+ caso + ": ")
        try:
            if int(elemento) > 0 :
                indice = int(elemento)
                elemento = lista[int(elemento) - 1]     
                inputCorrecto = True
            else:
                print("\nError. Introduce un numero positivo.")
        except:
            print("\nError. Introduce un numero correcto.") # Necesitamos numeros
    return indice

In [12]:
def ej1(): #Comentar e indexar para ej 1
    arbol = createTree()
    lista = displayAndList(arbol)
    indice = checkCorrectness(lista, "monumento")
    lista = list(arbol.iter("contenido"))
    arbolAux = lista[indice - 1]

    #for nodo in arbolAux.iter("atributo"): Esta comentado para el ej 3!!
    #       print (nodo.text)
    return arbolAux

Para la segunda parte se va a utilizar la API de la __EMT__. La __EMT__ dispone de un portal de datos abiertos que se puede encontrar en la página http://opendata.emtmadrid.es/Home. El objetivo del mismo es ofrecer diferentes tipos de datos sobre la actividad de los autobuses de la EMT. El acceso a los datos se realiza a través de una API de servicios web. 

Para poder utilizar la API de servicios, en primer lugar hay que registrarse, lo cual puede hacerse en la página http://opendata.emtmadrid.es/Formulario. El registro devuelve como resultado en un mensaje electrónico, dos valores:

* Id de cliente: Identificador de cliente

* Pass Key: Password

A continuación, hay que elegir el servicio que se quiere utilizar. Hay 4 servicios definidos: BUS, GEO, MEDIA, INFOPARKING y BICIMAD. Cada servicio tiene asociado un conjunto de métodos que al invocarlos, devuelven un resultado. Por ejemplo, el servicio BUS dispone del servicio __"GetRouteLines"__ que obtiene el itinerario de una línea (o varias líneas separadas por el carácter pipe(|)), con los vértices para construir las rectas del recorrido y las coordenadas UTM de los ejes viales y los códigos de parada. Si se quiere invocar desde Python, se puede hacer usando el siguiente código:

In [6]:
import requests
datos = {
    'idClient':'WEB.SERV.cbilbao@ucm.es',
    'PassKey':'4C22FDD0-EE62-4D7D-B1C8-629D6CB3E299',
    'SelectDate': '16/10/2018',
    'Lines':'27'
}
url = 'https://servicios.emtmadrid.es:8443/bus/servicebus.asmx/GetRouteLines'
response = requests.post(url, data = datos)
print(response.text)

<?xml version="1.0" encoding="utf-8"?>
<TABLA>
  <RESULTADO>0</RESULTADO>
  <DESCRIPCION>Resultado de la operacion Correcta</DESCRIPCION>
  <REG>
    <Line>027</Line>
    <SecDetail>10</SecDetail>
    <OrderDetail>1</OrderDetail>
    <Node>86</Node>
    <Distance>0</Distance>
    <DistStopPrev>0</DistStopPrev>
    <Name>Embajadores</Name>
    <PosxNode>440507,6</PosxNode>
    <PosyNode>4473192</PosyNode>
  </REG>
  <REG>
    <Line>027</Line>
    <SecDetail>10</SecDetail>
    <OrderDetail>1</OrderDetail>
    <Node>85</Node>
    <Distance>471</Distance>
    <DistStopPrev>471</DistStopPrev>
    <Name>Teatro Circo Price</Name>
    <PosxNode>440808</PosxNode>
    <PosyNode>4473251</PosyNode>
  </REG>
  <REG>
    <Line>027</Line>
    <SecDetail>10</SecDetail>
    <OrderDetail>1</OrderDetail>
    <Node>87</Node>
    <Distance>794</Distance>
    <DistStopPrev>323</DistStopPrev>
    <Name>José Antonio Armona</Name>
    <PosxNode>440988,5</PosxNode>
    <PosyNode>4473082</PosyNode>
  </REG>
  <R

En esta práctica se va a trabajar con varios métodos:

* Método __'GetStreet'__ del servicio GEO. Este método obtiene una lista de emplazamientos EMT coincidentes con una localización. Cada emplazamiento está compuesto por una lista de paradas EMT situadas dentro de un radio predefinido con todos sus atributos, así como las líneas EMT que pasan por cada parada de la lista. En el documento __Servicios_GEO.pdf__ adjunto a esta práctica, se explica con detalle los parámetros de entrada y el resultado que devuelve este método. El método se puede invocar indicando al menos el nombre de la calle y el número:

In [6]:
import requests

def getInfo(calle):
    datos = {
        'idClient':'WEB.SERV.cbilbao@ucm.es',
        'PassKey':'4C22FDD0-EE62-4D7D-B1C8-629D6CB3E299',
        'description': calle,
        'streetNumber':'',
        'Radius':'',
        'Stops':'',
        'statistics':'',
        'cultureInfo':''
    }

    url = 'https://servicios.emtmadrid.es:8443/geo/servicegeo.asmx/GetStreet'
    response = requests.post(url, data=datos)
    return response

El método devuelve un documento XML. Entre los elementos del documento, se encuentran las coordenadas X e Y del lugar buscado. En la búsqueda del ejemplo, la calle búscada tiene como coordenadas UTM X e Y:

* Método __'GetStreetRoute'__ del servicio MEDIA. Este método obtiene hasta tres rutas óptimas para ir de un sitio a otro a pie y en autobús. En el documento __Servicios_MEDIA.pdf__ adjunto a esta práctica, se explica con detalle los parámetros de entrada y el resultado que devuelve este método. El método se puede invocar indicando los siguientes parámetros

  * idClient: Código de cliente autorizado para la operación y suministrado por EMT.

  * passKey: Código de clave asociado al cliente.

  * coordinateXFrom: Campo para identificar la posición "x" del origen de la ruta a consultar. 
  
  * coordinateYFrom: Campo para identificar la posición "y" del origen de la ruta a consultar.

  * coordinateXTo: Campo para identificar la posición "x" del destino de la ruta a consultar. 
  
  * coordinateYTo: Campo para identificar la posición "y" del destino de la ruta a consultar. 

  * criteriaSelection: Campo que indica el criterio de la búsqueda. Los valores son:
  
       * 11 - Mínimo tiempo de trayecto
    
       * 13 - Mínimos trasbordos
    
       * 14 - Mínimo recorrido a pie

In [7]:
import requests
def getOptimal(x, y, x2, y2): #Ej 3
    datos = {
        'idClient':'WEB.SERV.cbilbao@ucm.es',
        'PassKey':'4C22FDD0-EE62-4D7D-B1C8-629D6CB3E299',
        'statistics':'',
        'cultureInfo':'',
        'coordinateXFrom': x,
        'coordinateYFrom':y,
        'originName':'',
        'coordinateXTo': x2,
        'coordinateYTo':y2,
        'destinationName':'',
        'criteriaSelection':'13',
        'day':'',
        'month':'',
        'year':'',
        'hour':'',
        'minute':'',
        'GenerarAudio':''
    }

    url = 'https://servicios.emtmadrid.es:8443/servicemedia/servicemedia.asmx/GetStreetRoute'
    response = requests.post(url, data=datos)
    return response

In [9]:
import requests
datos = {
'idClient':'WEB.SERV.cbilbao@ucm.es',
'PassKey':'4C22FDD0-EE62-4D7D-B1C8-629D6CB3E299',
'statistics':'',
'cultureInfo':'',
'coordinateXFrom': '-3,63271713087776',
'coordinateYFrom':'40,4543980802915',
'originName':'',
'coordinateXTo': '-3.7013019',
'coordinateYTo':'40.4176416',
'destinationName':'',
'criteriaSelection':'13',
'day':'',
'month':'',
'year':'',
'hour':'',
'minute':'',
'GenerarAudio':''
}
url = 'https://servicios.emtmadrid.es:8443/servicemedia/servicemedia.asmx/GetStreetRoute'
response = requests.post(url, data=datos)
print(response.text)

<?xml version="1.0" encoding="utf-8"?>
<root>
  <Resultado>
    <CodError>300</CodError>
    <Description>La recuperación de rutas ha sido correcta.</Description>
    <ListRouteData>
      <RouteData>
        <IdDescripcion>PLAZA LIMA-HORTALEZA * SOL-PLAZA PERU</IdDescripcion>
        <IdLinea>120 * 51</IdLinea>
        <DescriptionRouteData>
          <DescriptionDate>Fecha:18/10/2018</DescriptionDate>
          <DescriptionInitTime>Hora de Salida:10:30</DescriptionInitTime>
          <DescriptionEstimateTimeArrival>11:49</DescriptionEstimateTimeArrival>
          <Transfers>1</Transfers>
          <LongJourney>76</LongJourney>
        </DescriptionRouteData>
        <ListSectionRoute>
          <Section>
            <Order>0</Order>
            <WalkingLeg>
              <Distance>1236.2767685200442</Distance>
              <SourceWalkingLeg>
                <Name />
                <CoordinatesXY>
                  <X>-3,63234152</X>
                  <Y>40,45458872</Y>
            

El método devuelve un documento XML que contiene información sobre las rutas óptimas (se explica con detalle en el documento __Servicios_MEDIA.pdf__). En particular muestra la siguiente información:

* El elemento __DescriptionRouteData__ donde aparece la siguiente información:

  * DescriptionDate: Fecha de la ruta.
  * DescriptionInitTime: Descripción de hora de inicio de la ruta.
  * DescriptionEstimateTimeArrival: Descripción de la hora estimada de llegada.
  * Transfers: Trasbordos.
  * LongJourney: Duración del viaje.

* El elemento __ListSectionRoute__ aparece una lista de subelementos __Section__.Cada uno de ellos, describe parcialmente una parte de la ruta. En un caso ideal, aparecerá un subelemento de __Section__ de tipo __WalkingLeg__  que describe el inicio de la ruta andando, a continuación un conjunto de subelementos de __Section__ de tipo __BusLeg__ que describen cada uno de ellos la ruta en una línea de autobus que forma parte de la ruta buscada, y por último otro subelemento de __Section__ de tipo __WalkingLeg__  que describe el final de la ruta andando.

* El elemento __POI__ que muestra información sobre puntos de interés que se encuentran a lo largo de la ruta.

Se pide:

 __[Ejercicio 2 [2 puntos]]__

Crear una función que dada una calle de Madrid, devuelva las coordenadas X e Y de la calle.

 

In [4]:
from xml.etree import ElementTree
def coordenadas(calle):
    info = getInfo(calle)
    lista = []
    #requests.models.Response es el tipo de info
    file = open("response.xml", "w")
    file.write(info.text)
    file.close()
    file = open("response.xml", "r")
    arbol = ElementTree.parse(file)
    file.close()
    raiz = arbol.getroot()
    coordenadaX = raiz[0][7].text
    coordenadaY = raiz[0][8].text
    lista.append(coordenadaX)
    lista.append(coordenadaY)
    return lista


In [5]:
# Profesor José García Santesmases
calle = input("Calle de Madrid: ")
lista = coordenadas(calle)
print("Coordenada X", lista[0])
print("Coordenada Y", lista[1])


Calle de Madrid: Profesor José García Santesmases


NameError: name 'getInfo' is not defined

__[Ejercicio 3 [5 puntos]]__

Usando la función anterior completa el ejercicio 1. Para ello, se pide crear una función que solicite al usuario una calle de Madrid y obtenga la ruta más óptima para ir desde esa calle al monumento seleccionado por el usuario usando autobuses de la EMT. La función deberá imprimir por pantalla, la siguiente información:

 * Fecha de la ruta.
 
 * Hora de inicio de la ruta.
 
 * Hora estimada de llegada
 
 * Número de trasbordos.
 
 * Duración del viaje.
 
 * Descripción textual de la ruta.
 
 Usando el ejemplo de salida sería:
 
 * Fecha de la ruta: 26/10/2017
 
 * Hora de inicio de la ruta: 03:55
 
 * Hora estimada de llegada: 05:28
 
 * Número de trasbordos: 1
 
 * Duración del viaje: 93
 
 * Descripción textual de la ruta:
 
     * Caminar 42' hasta parada 3125 - EMILIO VARGAS-ARTURO SORIA, linea N4
     
     * 28' en autobús (línea N4) hasta parada 449 - SERRANO-ORTEGA Y GASSET
     
     * Desde parada 449 caminar 5' hasta parada 61 - Castellana-Ministerio Interior, linea N25
 
     * 15' en autobús (línea N25) hasta parada 3691 - SOL-SEVILLA

     * Desde parada 3691 caminar 3'  

#### Normas de entrega

* Fecha tope de entrega: 18/10/2018
* La entrega se realizará subiendo al campus virtual un notebook de Jupyter con la solución. El archivo tendrá como nombre FormatosII_GrupoX donde X será el número de grupo correspondiente.

In [18]:
calle = input("Calle de Madrid: ")
lista = coordenadas(calle)
x = lista[0]
y = lista[1]

#Conseguimos las coordenadas de ese monumento (x2,y2)
arbol = ej1() 

for nodo in arbol.iter(): 
    if nodo.attrib.get("nombre") == "COORDENADA-X":
        x2 = nodo.text
    if nodo.attrib.get("nombre") == "COORDENADA-Y":
        y2 = nodo.text

optimo = getOptimal(x, y, x2, y2)
file = open("response2.xml", "w")
file.write(optimo.text)
file.close()
file = open("response2.xml", "r")
arbol = ElementTree.parse(file)
file.close()
raiz = arbol.getroot()

print("Fecha de la ruta: ")
#DescriptionRouteData->DescriptionDate (ver Servicios_MEDIA.pdf)
for nodo in arbol.iter("Resultado"):
    print (nodo.tag)
    if nodo.tag == "DescriptionRouteData":
        print("Entra aqui")
        for nodo2 in nodo.iter():
            print (nodo2.text)
print("Hora de inicio de la ruta: ")
#DescriptionRouteData->DescriptionInitTime

print("Hora estimada de llegada: ")
#DescriptionRouteData->DescriptionEstimateTimeArrival

print("Número de trasbordos: ")
#DescriptionRouteData->Transfers

print("Duración del viaje: ")
#DescriptionRouteData->LongJourney

#Descripción textual de la ruta -> ListSectionRoute

Calle de Madrid: Profesor José García Santesmases
1	Banco de España
2	Basílica Pontificia de San Miguel
3	Biblioteca Nacional de España
4	Bolsa de Madrid
5	Casa de América
6	Casa de Cisneros
7	Casa de La Panadería
8	Casa de La Villa. Ayuntamiento de Madrid
9	Casa de Velázquez
10	Casón del Buen Retiro
11	Castillo de la Alameda
12	Catedral Santa María la Real de la Almudena
13	Centro de Estudios Políticos y Constitucionales
14	Centro Social y Cultural La Casa Encendida
15	Círculo de Bellas Artes
16	Conde Duque
17	Congreso de los Diputados
18	Consejo de Estado
19	Convento de las Comendadoras de Santiago
20	Edificio Metrópolis
21	Ermita de San Antonio de la Florida (Museo)
22	Ermita de San Isidro
23	Ermita de Santa María la Antigua
24	Ermita Virgen del Puerto
25	Estación ferroviaria Madrid Puerta de Atocha
26	Fundación BBVA. Palacio del Marqués de Salamanca
27	Galería de Cristal del Palacio de Cibeles
28	Iglesia Catedral de las Fuerzas Armadas
29	Iglesia Corpus Christi (Jerónimas Recoletas